# Imports

In [1]:
#Import of relevant libraries, classes or methods (in order of apperance)
import os
import pandas as pd
import numpy as np
from collections import Counter
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.base import clone
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import joblib
import shap
import json
import time
import pickle

In [2]:
# Get the current notebook directory
current_dir = os.getcwd()

# Go one level up and build the path to the data folder

data_path_df = os.path.join(current_dir, "..", "data", 
                         "kaggle_survey_2020_responses.csv")

data_path_encoder_assignment = os.path.join(current_dir, "..", "data", 
                         "encoder_assignment.csv")

data_path_unique_with_rank = os.path.join(current_dir, "..", "data", 
                         "unique_with_rank.csv")


# Load the CSV files and instantiate DataFrames

df                 = pd.read_csv(data_path_df,
                                 dtype = 'str')

encoder_assignment = pd.read_csv(data_path_encoder_assignment,
                                 sep = ';')


unique_with_rank = pd.read_csv(data_path_unique_with_rank, 
                               dtype = 'str',
                               sep = ';',
                               na_values=['nan', '', 'NaN'])

# Preparation

In [3]:
# Merge the question numbers with the question text for better readibility
 # We will shorten the column names later.

col_index = df.columns.tolist()
# print(col_index)

questions = df.iloc[0,:].tolist()
# print(questions)

# Generate the composed index in a new list. Use a list comprehenshion with built in zip function:
col_index_new = [f"{col}_{val}" for col, val in zip(col_index, questions)]

#  print(col_index_new)

# Step 2: Assign the new index to the DataFrame
df.columns = col_index_new

# Step 3: Delete the first row (no longer used)
df = df.iloc[1:,]

In [4]:
###---EXPORT FOR STREMLIT---###
# Get the current notebook directory
current_dir = os.getcwd()

# Build the path to the 'downloads_for_streamlit' folder
export_path = os.path.join(current_dir, "downloads_for_streamlit", "df_long.csv")

# Export the DataFrame as CSV
df.to_csv(export_path, sep=';', index=False)

# Test output
print(f"CSV file successfully saved to: {export_path}")
###---EXPORT FOR STREMLIT---###

CSV file successfully saved to: C:\Users\Simon\Documents\GitHub\RoleRecommender\final_report\notebooks\downloads_for_streamlit\df_long.csv


In [5]:
# Within the 'Selected Choice'-columns, it is better to transform the values
# into Booleans, because they are better to count and better to modell.
# All others are strings. An NaN is interpreted as "z_not selected"

mark_bool = '- Selected Choice -'                     # If the questions have "- Seleceted choice - ", the answer ist yes or no (Boolean)


for col in df.columns:                                # NaNs are interpreted as False
    if mark_bool in col:
        df[col] = df[col].apply(lambda x:
                                False if pd.isna(x)
                                else True)            # If it's not a NaN, the answer is given. We replace it with a True
    else:
        df[col] = df[col].fillna('z_Not selected')     # NaNs are interpreted as "z_Not selected". z as a prefix to secure that it is always on the right hand side of the plot.
        df[col] = df[col].astype(str)                  # The whole columns is transformed into a string

In [6]:
# We now choose the questions of boolean type and assign them do a DataFram df_bool::

# Questions with boolean character:

col_bool = ['^Q7', '^Q9', '^Q14', '^Q16', '^Q17', '^Q18', '^Q19',
            '^Q26_A', '^Q27_A', '^Q28_A', '^Q29_A', '^Q31_A', '^Q33_A', '^Q34_A', '^Q35_A', '^Q37']

# Filter rules
regex_pattern_bool = '|'.join(col_bool)

# Filter all Boolean columns
df_bool = df.filter(regex=regex_pattern_bool)

In [7]:
# Define a pattern to sum up the relevant columns

# Mapping Regex → Columns for summing up
sum_columns = {
    '^Q7':    'Q7_No. of Regular used programming languages?',
    '^Q9':    'Q9_No. of Specialized IDE?',
    '^Q14':   'Q14_No. of DataViz Libs or Tools?',
    '^Q16':   'Q16_No. of ML Framworks?',
    '^Q17':   'Q17_No. of ML algorithms?',
    '^Q18':   'Q18_No. of Computer Vsion methods?',
    '^Q19':   'Q19_No. of NLP methods?',
    '^Q26_A': 'Q26_A No. of Current Cloud platforms?',
    '^Q27_A': 'Q27_A No. of Current Cloud Products?',
    '^Q28_A': 'Q28_A No. of Current ML products?',
    '^Q29_A': 'Q29_A No. of Big Data Tools?',
    '^Q31_A': 'Q31_A No. of BI Tools used?',
    '^Q33_A': 'Q33_A No. of Automted ML Tools?',
    '^Q34_A': 'Q34_A No. of Auto ML Tools?',
    '^Q35_A': 'Q35_A No. of ML Experiment Management?',
    '^Q37':   'Q37_No. of Learning Platforms?'

}

for regex, new_col in sum_columns.items():
    matching_cols = df.filter(regex=regex).columns                                         # Select all columns that match
    valid_cols = [col for col in matching_cols if "- Selected Choice - None" not in col]   # Exclude columns with "none"
    df_bool[new_col] = df[valid_cols].sum(axis=1)                                          # sum up valid columns


C:\Users\Simon\AppData\Local\Temp\ipykernel_12448\178850794.py:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_bool[new_col] = df[valid_cols].sum(axis=1)                                          # sum up valid columns
C:\Users\Simon\AppData\Local\Temp\ipykernel_12448\178850794.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bool[new_col] = df[valid_cols].sum(axis=1)                                          # sum up valid columns
C:\Users\Simon\AppData\Local\Temp\ipykernel_12448\178850794.py:27: Performanc

In [8]:
# We now choose the questions of string type (multiple choice) and assign them do a DataFram df_string::

col_string = ['Q1_What is your age (# years)?',
              'Q4_What is the highest level of formal education that you have attained or plan to attain within the next 2 years?',
              'Q5_Select the title most similar to your current role (or most recent title if retired): - Selected Choice',
              'Q6_For how many years have you been writing code and/or programming?',
              'Q8_What programming language would you recommend an aspiring data scientist to learn first? - Selected Choice',
              'Q13_Approximately how many times have you used a TPU (tensor processing unit)?',
              'Q15_For how many years have you used machine learning methods?',
              'Q20_What is the size of the company where you are employed?',
              'Q22_Does your current employer incorporate machine learning methods into their business?',
              'Q24_What is your current yearly compensation (approximate $USD)?',
              'Q30_Which of the following big data products (relational database, data warehouse, data lake, or similar) do you use most often? - Selected Choice',
              'Q32_Which of the following business intelligence tools do you use most often? - Selected Choice']

# Filter rules
regex_pattern_str = '|'.join(col_string)

# Filter all Boolean columns
df_string = df[[col for col in col_string if col in df.columns]]

In [9]:
# OK, this is confusing but effective code. To be corrected later.

# Q5_Select the title most similar to your current role
old_name = df_string.columns[2]
new_name = "Q5_Select the title most similar to your current role"
df_string.rename(columns={old_name: new_name}, inplace=True)

# Q8_What programming language would you recommend an aspiring data scientist to learn first?
old_name = df_string.columns[4]
new_name = "Q8_What programming language would you recommend an aspiring data scientist to learn first?"
df_string.rename(columns={old_name: new_name}, inplace=True)

# Q30_Which of the following big data products (relational database, data warehouse, data lake, or similar) do you use most often?
old_name = df_string.columns[10]
new_name = "Q30_Which of the following big data products (relational database, data warehouse, data lake, or similar) do you use most often?"
df_string.rename(columns={old_name: new_name}, inplace=True)

# Q32_Which of the following business intelligence tools do you use most often?
old_name = df_string.columns[11]
new_name = 'Q32_Which of the following business intelligence tools do you use most often?'
df_string.rename(columns={old_name: new_name}, inplace=True)

C:\Users\Simon\AppData\Local\Temp\ipykernel_12448\585003900.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_string.rename(columns={old_name: new_name}, inplace=True)
C:\Users\Simon\AppData\Local\Temp\ipykernel_12448\585003900.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_string.rename(columns={old_name: new_name}, inplace=True)
C:\Users\Simon\AppData\Local\Temp\ipykernel_12448\585003900.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-

In [10]:
# We merge qustions with boolean and string type into one DataFrame, then we drop
# columns with "- Selceted - " answers.

df_short = pd.concat([df_string, df_bool], axis=1)
df_short = df_short.drop(columns=df_short.filter(regex='Selected').columns)
df_short.columns

Index(['Q1_What is your age (# years)?',
       'Q4_What is the highest level of formal education that you have attained or plan to attain within the next 2 years?',
       'Q5_Select the title most similar to your current role',
       'Q6_For how many years have you been writing code and/or programming?',
       'Q8_What programming language would you recommend an aspiring data scientist to learn first?',
       'Q13_Approximately how many times have you used a TPU (tensor processing unit)?',
       'Q15_For how many years have you used machine learning methods?',
       'Q20_What is the size of the company where you are employed?',
       'Q22_Does your current employer incorporate machine learning methods into their business?',
       'Q24_What is your current yearly compensation (approximate $USD)?',
       'Q30_Which of the following big data products (relational database, data warehouse, data lake, or similar) do you use most often?',
       'Q32_Which of the following business 

In [11]:
# We delete all rows with entries ['Other', 'Student', 'z_Not selected', 'Currently not employed ']
df_heat = df_short[~df_short['Q5_Select the title most similar to your current role'].isin(
    ['Other', 'Student', 'z_Not selected', 'Currently not employed'])]

In [12]:
# Preparing Columns transformation

## Assign colums to encoder type:

columns_to_keep = [col for col in encoder_assignment["column"] if col in df_short.columns]
lab_columns  = encoder_assignment.query("encoder == 'lab'")["column"].tolist()
ohe_columns  = encoder_assignment.query("encoder == 'ohe'")["column"].tolist()
ord_columns  = encoder_assignment.query("encoder == 'ord'")["column"].tolist()


## Generate order for categories in OrderEncoder
### Initial list


unique_with_rank.columns = unique_with_rank.columns.str.replace(r'\s+', ' ',
                                                                regex=True).str.strip()
### Clean list from NaNs

def clean_float_strings(val):
    # Werte, die echte NaN darstellen sollen
    if val in ['nan', 'NaN', '']:
        return np.nan
    try:
        f = float(val)
        if f.is_integer():
            return str(int(f))  #  '0.0' → '0'
        return str(f)           #  '2.5' bleibt '2.5'
    except:
        return val             # Text remains text

unique_with_rank_cl = unique_with_rank.applymap(clean_float_strings)

### Create finale list of ordered categories for OrdinalEncoder:

unique_dict = {}

for col in unique_with_rank_cl.columns:
    cats = unique_with_rank_cl[col].dropna().unique().tolist()   # Extract all values without NaNa and remove duplicates
    cats = [cat for cat in cats if cat != 'nan']                 # Remove nan as string
    unique_dict[col] = cats

### Final list for Ordninal Encoder

categories = list(unique_dict.values())

C:\Users\Simon\AppData\Local\Temp\ipykernel_12448\4195297110.py:31: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  unique_with_rank_cl = unique_with_rank.applymap(clean_float_strings)


In [13]:
###---EXPORT FOR STREAMLIT---###
# Get the current notebook directory
current_dir = os.getcwd()

# Build the path to the 'downloads_for_streamlit' folder
export_path = os.path.join(current_dir, "downloads_for_streamlit", "categories.json")

# Save the list as JSON
with open(export_path, 'w', encoding='utf-8') as f:
    json.dump(categories, f, ensure_ascii=False, indent=4)

# Test output
print(f"JSON file successfully saved to: {export_path}")
###---EXPORT FOR STREAMLIT---###

JSON file successfully saved to: C:\Users\Simon\Documents\GitHub\RoleRecommender\final_report\notebooks\downloads_for_streamlit\categories.json


In [14]:
###---EXPORT FOR STREAMLIT---###
# Get the current notebook directory
current_dir = os.getcwd()

# Build the path to the 'downloads_for_streamlit' folder
export_path = os.path.join(current_dir, "downloads_for_streamlit", "ohe_columns.json")

# Save the list as JSON
with open(export_path, 'w', encoding='utf-8') as f:
    json.dump(ohe_columns, f, ensure_ascii=False, indent=4)

# Test output
print(f"JSON file successfully saved to: {export_path}")
###---EXPORT FOR STREAMLIT---###


JSON file successfully saved to: C:\Users\Simon\Documents\GitHub\RoleRecommender\final_report\notebooks\downloads_for_streamlit\ohe_columns.json


In [15]:
###---EXPORT FOR STREAMLIT---###
# Get the current notebook directory
current_dir = os.getcwd()

# Build the path to the 'downloads_for_streamlit' folder
export_path = os.path.join(current_dir, "downloads_for_streamlit", "ord_columns.json")

# Save the list as JSON
with open(export_path, 'w', encoding='utf-8') as f:
    json.dump(ord_columns, f, ensure_ascii=False, indent=4)

# Test output
print(f"JSON file successfully saved to: {export_path}")
###---EXPORT FOR STREAMLIT---###

JSON file successfully saved to: C:\Users\Simon\Documents\GitHub\RoleRecommender\final_report\notebooks\downloads_for_streamlit\ord_columns.json


# Generate Data Sets

In [16]:
# Data Set S for classical Data Science Roles vs. Software Engeneering

## We now reduce the data set to the above mentioned roles only:
selected_roles_S = [
     'Data Scientist',
        'Software Engineer',
        'Data Analyst'
        ]

df_heat_S = df_heat.loc[
    df_heat.iloc[:, 2].isin(selected_roles_S)
    ]

## Eliminate the target variables. X represents the matrix of explanatory variables for the ML model.
X_S = df_heat_S.drop(lab_columns, axis=1)

## Define y as the target for the ML model.
y_S = df_heat_S[lab_columns[0]]

## Split Train- and Test-Set:
X_train_S, X_test_S, y_train_S, y_test_S = train_test_split(X_S,
                                                            y_S,
                                                            test_size=0.2,
                                                            random_state=42)

In [17]:
###---EXPORT FOR STREAMLIT---###
# Get the current notebook directory
current_dir = os.getcwd()

# Build the path to the 'downloads_for_streamlit' folder
export_path = os.path.join(current_dir, "downloads_for_streamlit", "df_heat_S.csv")

# Export the DataFrame as CSV
df_heat_S.to_csv(export_path, sep=';', index=False)

# Test output
print(f"CSV file successfully saved to: {export_path}")
###---EXPORT FOR STREAMLIT---###

CSV file successfully saved to: C:\Users\Simon\Documents\GitHub\RoleRecommender\final_report\notebooks\downloads_for_streamlit\df_heat_S.csv


In [18]:
###---EXPORT FOR STREAMLIT---###
# Get the current notebook directory
current_dir = os.getcwd()

# Build the path to the 'downloads_for_streamlit' folder
export_path = os.path.join(current_dir, "downloads_for_streamlit", "default_X_train_S.csv")

# Create the default_X_train_S DataFrame
default_X_train_S = X_train_S.mode().iloc[0:1]

# Export the DataFrame as CSV
default_X_train_S.to_csv(export_path, sep=';', index=False, encoding="utf-8-sig")

# Test output
print(f"CSV file successfully saved to: {export_path}")
###---EXPORT FOR STREAMLIT---###


CSV file successfully saved to: C:\Users\Simon\Documents\GitHub\RoleRecommender\final_report\notebooks\downloads_for_streamlit\default_X_train_S.csv


In [19]:
###---EXPORT FOR STREAMLIT (S)---###
# Get the current notebook directory
current_dir = os.getcwd()

# Build the path to the 'downloads_for_streamlit' folder
export_path = os.path.join(current_dir, "downloads_for_streamlit", "unique_values_per_feature_S.json")

# Generate the dictionary of unique values
def extract_unique_values(df):
    return {col: sorted(df[col].dropna().unique().astype(str).tolist()) for col in df.columns}

unique_map_S = extract_unique_values(df_heat_S)

# Save as JSON
with open(export_path, "w", encoding="utf-8-sig") as f:
    json.dump(unique_map_S, f, ensure_ascii=False, indent=2)

# Test output
print(f"JSON file successfully saved to: {export_path}")
###---EXPORT FOR STREAMLIT (S)---###

JSON file successfully saved to: C:\Users\Simon\Documents\GitHub\RoleRecommender\final_report\notebooks\downloads_for_streamlit\unique_values_per_feature_S.json


In [20]:
# Data Set L for for Carrer paths Data Science vx. Tech

## List of job roles we want to include in the filtered DataFrame:
selected_roles_L = [
    'Data Scientist',
    'Software Engineer',
    'Data Analyst',
    'Research Scientist',
    'Machine Learning Engineer',
    'Data Engineer',
    'DBA/Database Engineer'
]

## Re-arragen the target columns such that we include the career path:
role_column = 'Q5_Select the title most similar to your current role'
df_heat_L = df_heat[df_heat[role_column].isin(selected_roles_L)]                 # Filter the original DataFrame to only include the selected roles
def role_DS(dframe, col_select):                                                 # Function to classify each role into a broader role group: DS, Tech, or Business
    ds_roles = [
        'Data Scientist',
        'Data Analyst',
        'Machine Learning Engineer',
        'Data Engineer',
        'Research Scientist'
    ]
    tech_roles = [
        'Software Engineer',
        'DBA/Database Engineer'
    ]
    def map_role(role):                                                          # Internal helper function to map job title to role group
      if role in ds_roles:
        return 'DS'
      elif role in tech_roles:
        return 'Tech'
      else:
        return 'Other'
    return dframe[col_select].apply(map_role)                                    # Apply the role mapping to the selected column


df_heat_L['role_group'] = role_DS(df_heat_L, role_column)                        # Add a new column to the DataFrame with the role group classification
df_heat_L = df_heat_L.drop(columns=[role_column])                                # Drop the old columns and ...
col_to_insert = df_heat_L.pop('role_group')                                      # ... insert the new one.
df_heat_L.insert(2, role_column, col_to_insert)


## Eliminate the target variables. X represents the matrix of explanatory variables for the ML model.
X_L = df_heat_L.drop(lab_columns, axis=1)

# Define y as the target for the ML model.
y_L = df_heat_L[lab_columns[0]]

## Split Train- and Test-Set:
X_train_L, X_test_L, y_train_L, y_test_L = train_test_split(X_L,
                                                            y_L,
                                                            test_size=0.2,
                                                            random_state=42)

C:\Users\Simon\AppData\Local\Temp\ipykernel_12448\134686113.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_heat_L['role_group'] = role_DS(df_heat_L, role_column)                        # Add a new column to the DataFrame with the role group classification


In [21]:
###---EXPORT FOR STREAMLIT---###
# Get the current notebook directory
current_dir = os.getcwd()

# Build the path to the 'downloads_for_streamlit' folder
export_path = os.path.join(current_dir, "downloads_for_streamlit", "df_heat_L.csv")

# Export the DataFrame as CSV
df_heat_L.to_csv(export_path, sep=';', index=False)

# Test output
print(f"CSV file successfully saved to: {export_path}")
###---EXPORT FOR STREAMLIT---###

CSV file successfully saved to: C:\Users\Simon\Documents\GitHub\RoleRecommender\final_report\notebooks\downloads_for_streamlit\df_heat_L.csv


In [22]:
###---EXPORT FOR STREAMLIT---###
# Get the current notebook directory
current_dir = os.getcwd()

# Build the path to the 'downloads_for_streamlit' folder
export_path = os.path.join(current_dir, "downloads_for_streamlit", "default_X_train_L.csv")

# Create the default_X_train_L DataFrame
default_X_train_L = X_train_L.mode().iloc[0:1]

# Export the DataFrame as CSV
default_X_train_L.to_csv(export_path, sep=';', index=False, encoding="utf-8-sig")

# Test output
print(f"CSV file successfully saved to: {export_path}")
###---EXPORT FOR STREAMLIT---###

CSV file successfully saved to: C:\Users\Simon\Documents\GitHub\RoleRecommender\final_report\notebooks\downloads_for_streamlit\default_X_train_L.csv


In [23]:
###---EXPORT FOR STREAMLIT (L)---###
# Get the current notebook directory
current_dir = os.getcwd()

# Build the path to the 'downloads_for_streamlit' folder
export_path = os.path.join(current_dir, "downloads_for_streamlit", "unique_values_per_feature_L.json")

# Generate the dictionary of unique values
def extract_unique_values(df):
    return {col: sorted(df[col].dropna().unique().astype(str).tolist()) for col in df.columns}

unique_map_L = extract_unique_values(df_heat_L)

# Save as JSON
with open(export_path, "w", encoding="utf-8-sig") as f:
    json.dump(unique_map_L, f, ensure_ascii=False, indent=2)

# Test output
print(f"JSON file successfully saved to: {export_path}")
###---EXPORT FOR STREAMLIT (L)---###

JSON file successfully saved to: C:\Users\Simon\Documents\GitHub\RoleRecommender\final_report\notebooks\downloads_for_streamlit\unique_values_per_feature_L.json


# Data preprocessing

In [24]:
# Apply Label Encoder for target

lab_L = LabelEncoder()
lab_S = LabelEncoder()


y_train_L = pd.Series(y_train_L)
y_test_L = pd.Series(y_test_L)
y_train_L = lab_L.fit_transform(y_train_L)
y_test_L = lab_L.transform(y_test_L)

y_train_S = pd.Series(y_train_S)
y_test_S = pd.Series(y_test_S)
y_train_S = lab_S.fit_transform(y_train_S)
y_test_S = lab_S.transform(y_test_S)


# Backward Transformation of targets of L
y_train_L_original_labels = lab_L.inverse_transform(y_train_L)
y_test_L_original_labels = lab_L.inverse_transform(y_test_L)


# Backward Transformation of targets of S
y_train_S_original_labels = lab_S.inverse_transform(y_train_S)
y_test_S_original_labels = lab_S.inverse_transform(y_test_S)

In [25]:
ohe_transformer = Pipeline([
    ('ohe',   OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'))
])

ord_transformer = Pipeline([

    ('ord', OrdinalEncoder(categories=categories, handle_unknown='use_encoded_value', unknown_value=-1))
])


# Define preprocessors separately for S and L
preprocessor_S = ColumnTransformer([
    ('ohe', ohe_transformer, ohe_columns),
    ('ord', ord_transformer, ord_columns)
])

preprocessor_L = ColumnTransformer([
    ('ohe', ohe_transformer, ohe_columns),
    ('ord', ord_transformer, ord_columns)
])

# Final ML Model: Optimized XGBoost

## Code

In [26]:
# Determine the number of unique classes in each target set
num_classes_S = len(np.unique(y_train_S))
num_classes_L = len(np.unique(y_train_L))

# Build pipelines, each with their own preprocessor

# Pipeline for dataset S
pipe_xgb_S = Pipeline([
    ('preprocessing', preprocessor_S),     # Uses the dedicated preprocessor for S
    ('xgb', XGBClassifier(
        objective='multi:softmax',
        num_class=num_classes_S,           # Set according to unique classes in y_train_S
        eval_metric='mlogloss',
        random_state=42
    ))
])

# Pipeline for dataset L
pipe_xgb_L = Pipeline([
    ('preprocessing', preprocessor_L),     # Uses the dedicated preprocessor for L
    ('xgb', XGBClassifier(
        objective='multi:softmax',
        num_class=num_classes_L,           # Set according to unique classes in y_train_L
        eval_metric='mlogloss',
        random_state=42
    ))
])


# Define the optimal hyperparameters found for Dataset S through Randomized Search
best_params_S = {
    'xgb__reg_lambda': 1.0,         # L2 regularization term
    'xgb__n_estimators': 200,       # Number of boosting rounds (trees)
    'xgb__min_child_weight': 10,    # Minimum sum of instance weight (hessian) needed in a child
    'xgb__max_depth': 3,            # Maximum depth of a tree
    'xgb__learning_rate': 0.1       # Step size shrinkage to prevent overfitting
}

# Define the optimal hyperparameters found for Dataset L through Randomized Search
best_params_L = {
    'xgb__reg_lambda': 2.0,
    'xgb__n_estimators': 300,
    'xgb__min_child_weight': 20,
    'xgb__max_depth': 6,
    'xgb__learning_rate': 0.1
}

# Set optimized hyperparameters (from Randomized Search or Grid Search) for each pipeline
pipe_xgb_S.set_params(**best_params_S)
pipe_xgb_L.set_params(**best_params_L)

# Fit/train both pipelines with their respective training data
pipe_xgb_S.fit(X_train_S, y_train_S)
pipe_xgb_L.fit(X_train_L, y_train_L)

# #---Predict and evaluate---#

y_xgb_S = pipe_xgb_S.predict(X_test_S)
y_xgb_L = pipe_xgb_L.predict(X_test_L)

print("Results for data set S with XGBoost (Data Science Roles)\n", classification_report(y_test_S, y_xgb_S, target_names=lab_S.classes_, digits=4))
print("\nResults for data set L  With XGBoost (General career paths)\n", classification_report(y_test_L, y_xgb_L, target_names=lab_L.classes_, digits=4))


proba_S = pipe_xgb_S.predict_proba(X_test_S)  # returns probabilities for S classes
proba_L = pipe_xgb_L.predict_proba(X_test_L)  # returns probabilities for L classes

Results for data set S with XGBoost (Data Science Roles)
                    precision    recall  f1-score   support

     Data Analyst     0.5907    0.4651    0.5204       301
   Data Scientist     0.6850    0.7538    0.7178       528
Software Engineer     0.6872    0.7063    0.6966       395

         accuracy                         0.6675      1224
        macro avg     0.6543    0.6417    0.6449      1224
     weighted avg     0.6625    0.6675    0.6624      1224


Results for data set L  With XGBoost (General career paths)
               precision    recall  f1-score   support

          DS     0.8433    0.9160    0.8782      1369
        Tech     0.6179    0.4439    0.5167       419

    accuracy                         0.8054      1788
   macro avg     0.7306    0.6800    0.6974      1788
weighted avg     0.7905    0.8054    0.7934      1788



C:\Users\Simon\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Simon\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [27]:
###---EXPORT FOR STREAMLIT---###
# Get the current notebook directory
current_dir = os.getcwd()

# Build the export paths
export_path_L = os.path.join(current_dir, "downloads_for_streamlit", "pipe_xgb_L.pkl")
export_path_S = os.path.join(current_dir, "downloads_for_streamlit", "pipe_xgb_S.pkl")

# Save pipeline for y_L
joblib.dump(pipe_xgb_L, export_path_L)
print(f"Pipeline L saved to: {export_path_L}")

# Save pipeline for y_S
joblib.dump(pipe_xgb_S, export_path_S)
print(f"Pipeline S saved to: {export_path_S}")
###---EXPORT FOR STREAMLIT---###

Pipeline L saved to: C:\Users\Simon\Documents\GitHub\RoleRecommender\final_report\notebooks\downloads_for_streamlit\pipe_xgb_L.pkl
Pipeline S saved to: C:\Users\Simon\Documents\GitHub\RoleRecommender\final_report\notebooks\downloads_for_streamlit\pipe_xgb_S.pkl


In [28]:
###---EXPORT FOR STREAMLIT---###
# Get the current notebook directory
current_dir = os.getcwd()

# Build export paths
export_path_S = os.path.join(current_dir, "downloads_for_streamlit", "classification_report_S.json")
export_path_L = os.path.join(current_dir, "downloads_for_streamlit", "classification_report_L.json")

# Generate Classification Reports
report_S = classification_report(y_test_S, y_xgb_S, output_dict=True)
report_L = classification_report(y_test_L, y_xgb_L, output_dict=True)

# Save as JSON
with open(export_path_S, "w") as f:
    json.dump(report_S, f, indent=4)

with open(export_path_L, "w") as f:
    json.dump(report_L, f, indent=4)

# Test output
print(f"Classification report S saved to: {export_path_S}")
print(f"Classification report L saved to: {export_path_L}")
###---EXPORT FOR STREAMLIT---###

Classification report S saved to: C:\Users\Simon\Documents\GitHub\RoleRecommender\final_report\notebooks\downloads_for_streamlit\classification_report_S.json
Classification report L saved to: C:\Users\Simon\Documents\GitHub\RoleRecommender\final_report\notebooks\downloads_for_streamlit\classification_report_L.json


In [29]:
###---EXPORT FOR STREAMLIT---###
# Get the current notebook directory
current_dir = os.getcwd()

# Build export paths
export_path_S = os.path.join(current_dir, "downloads_for_streamlit", "confusion_matrix_S.json")
export_path_L = os.path.join(current_dir, "downloads_for_streamlit", "confusion_matrix_L.json")

# --- Calculate Confusion Matrices ---
cm_S = confusion_matrix(y_test_S, y_xgb_S).tolist()  # .tolist() converts NumPy array to nested list
cm_L = confusion_matrix(y_test_L, y_xgb_L).tolist()

# --- Optional: Label axes ---
confusion_S = {
    "labels": list(lab_S.classes_),
    "matrix": cm_S
}

confusion_L = {
    "labels": list(lab_L.classes_),
    "matrix": cm_L
}

# --- Save as JSON files ---
with open(export_path_S, "w") as f:
    json.dump(confusion_S, f, indent=4)

with open(export_path_L, "w") as f:
    json.dump(confusion_L, f, indent=4)

# Test output
print(f"Confusion matrix S saved to: {export_path_S}")
print(f"Confusion matrix L saved to: {export_path_L}")
###---EXPORT FOR STREAMLIT---###


Confusion matrix S saved to: C:\Users\Simon\Documents\GitHub\RoleRecommender\final_report\notebooks\downloads_for_streamlit\confusion_matrix_S.json
Confusion matrix L saved to: C:\Users\Simon\Documents\GitHub\RoleRecommender\final_report\notebooks\downloads_for_streamlit\confusion_matrix_L.json


## Feature Importance with SHAP

In [30]:
# This is code from Gemini, I admit. We use it for getting the correct feature names from the original DataFrame:

def get_feature_names(column_transformer):
    feature_names = []

    # Iterate over each transformer in the ColumnTransformer
    for name, transformer, columns in column_transformer.transformers_:

        # Skip the 'remainder' placeholder for now
        if name != 'remainder':

            # If the transformer is a Pipeline, get the last step (usually the actual transformer)
            if hasattr(transformer, 'named_steps'):
                transformer = list(transformer.named_steps.values())[-1]

            # Try to get feature names using the modern sklearn method
            if hasattr(transformer, 'get_feature_names_out'):
                names = transformer.get_feature_names_out(columns)

            # Fall back to older sklearn versions' method if needed
            elif hasattr(transformer, 'get_feature_names'):
                names = transformer.get_feature_names(columns)

            # If the transformer does not provide feature names, use the input column names/indices as is
            else:
                names = columns

            # Add the extracted feature names to the final list
            feature_names.extend(names)

        # If this transformer is the 'remainder' part of the ColumnTransformer
        else:
            # If remainder is set to 'passthrough', add those columns as feature names directly
            if transformer == 'passthrough':
                feature_names.extend(columns)

    # Return the full list of feature names after processing all transformers
    return feature_names

In [31]:
def shap_analysis_from_pipeline(pipe, X_train, filename_csv):
    # Extract model and preprocessor from pipeline
    model = pipe.named_steps['xgb']
    preprocessor = pipe.named_steps['preprocessing']

    # Sample and preprocess data
    X_sample = X_train.sample(100, random_state=42)
    X_transformed = preprocessor.transform(X_sample)

    # Get feature names
    feature_names = get_feature_names(preprocessor)

    # Initialize SHAP explainer and calculate SHAP values
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X_transformed)  # shape: (samples, features, classes)

    # Get class names
    class_names = model.classes_

    # Rearrange SHAP values for class-wise iteration
    shap_values_classwise = np.transpose(shap_values, (2, 0, 1))  # (classes, samples, features)

    # Collect mean absolute SHAP values per class
    shap_list = []
    for i, class_name in enumerate(class_names):
        df_shap = pd.DataFrame(shap_values_classwise[i], columns=feature_names)
        shap_mean = df_shap.abs().mean().sort_values(ascending=False)
        shap_list.append(shap_mean.rename(f"Mean |SHAP| ({class_name})"))

    # Combine results into one DataFrame
    df_shap_all = pd.concat(shap_list, axis=1)

    # Display the combined table
    display(df_shap_all)

    # Save to CSV
    df_shap_all.to_csv(filename_csv, sep = ';') # <-- Dies ist dein ursprünglicher Speichercode

    # Plot SHAP summary bar chart
   # shap.summary_plot(shap_values, X_transformed, feature_names=feature_names, plot_type="bar")

    return df_shap_all # Saves the DataFrame for later download

In [32]:
# Call for model L, and capture the returned DataFrame
df_shap_all_L = shap_analysis_from_pipeline(
    pipe_xgb_L, X_train_L,
    os.path.join(os.getcwd(), "downloads_for_streamlit", "shap_feature_importance_all_classes_L.csv")
)

# Call for model S, and capture the returned DataFrame
df_shap_all_S = shap_analysis_from_pipeline(
    pipe_xgb_S, X_train_S,
    os.path.join(os.getcwd(), "downloads_for_streamlit", "shap_feature_importance_all_classes_S.csv")
)

###---EXPORT FOR STREAMLIT---###
# Get the current notebook directory
current_dir = os.getcwd()

# Build export paths
export_path_L = os.path.join(current_dir, "downloads_for_streamlit", "shap_feature_importance_all_classes_L.csv")
export_path_S = os.path.join(current_dir, "downloads_for_streamlit", "shap_feature_importance_all_classes_S.csv")

# Test output
print(f"SHAP feature importance L saved to: {export_path_L}")
print(f"SHAP feature importance S saved to: {export_path_S}")
###---EXPORT FOR STREAMLIT---###

,Mean |SHAP| (0),Mean |SHAP| (1)
Q6_For how many years have you been writing code and/or programming?,0.317964,0.317964
Q4_What is the highest level of formal education that you have attained or plan to attain within the next 2 years?,0.246095,0.246095
Q7_No. of Regular used programming languages?,0.222353,0.222353
Q15_For how many years have you used machine learning methods?,0.207601,0.207601
Q14_No. of DataViz Libs or Tools?,0.139828,0.139828
...,...,...
Q32_Which of the following business intelligence tools do you use most often?_SAP Analytics Cloud,0.000000,0.000000
Q32_Which of the following business intelligence tools do you use most often?_Qlik,0.000000,0.000000
Q32_Which of the following business intelligence tools do you use most often?_Other,0.000000,0.000000
Q32_Which of the following business intelligence tools do you use most often?_Looker,0.000000,0.000000


,Mean |SHAP| (0),Mean |SHAP| (1),Mean |SHAP| (2)
Q6_For how many years have you been writing code and/or programming?,0.319807,0.145118,0.449246
Q31_A No. of BI Tools used?,0.196799,0.006945,0.247064
Q18_No. of Computer Vsion methods?,0.173698,0.037402,0.138553
Q27_A No. of Current Cloud Products?,0.145569,0.028404,0.041869
Q37_No. of Learning Platforms?,0.120146,0.040045,0.026904
...,...,...,...
Q32_Which of the following business intelligence tools do you use most often?_Domo,0.000000,0.000000,0.000000
Q32_Which of the following business intelligence tools do you use most often?_Looker,0.000000,0.000000,0.000000
Q32_Which of the following business intelligence tools do you use most often?_Microsoft Power BI,0.000000,0.000000,0.000000
Q32_Which of the following business intelligence tools do you use most often?_Other,0.000000,0.000000,0.000000


SHAP feature importance L saved to: C:\Users\Simon\Documents\GitHub\RoleRecommender\final_report\notebooks\downloads_for_streamlit\shap_feature_importance_all_classes_L.csv
SHAP feature importance S saved to: C:\Users\Simon\Documents\GitHub\RoleRecommender\final_report\notebooks\downloads_for_streamlit\shap_feature_importance_all_classes_S.csv
